<a href="https://colab.research.google.com/github/Alvise84/Computer_Vision/blob/main/Digit_Reconstruction_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Digit Reconstruction VAE**

## **Описание проекта**

Цель данного проекта — создание вариационного автокодировщика (VAE) для реконструкции цифр из датасета MNIST. Модель будет обучаться на бинаризованных изображениях цифр и способна будет восстанавливать исходные изображения из латентного представления. Кроме того, модель позволит исследовать латентное пространство и генерировать новые цифры.

## **Примеры применения**

**Обработка изображений:** Модель может быть использована для очистки и восстановления поврежденных изображений цифр.

**Генерация данных:** Для создания синтетических данных, которые могут быть использованы в задачах обучения машин и улучшении моделей.

**Исследование латентного пространства:** Анализ структуры латентного пространства может помочь понять, как модель представляет различные цифры.

**Компрессия данных:** Модель может использоваться для сжатия и хранения изображений цифр в компактной форме.

**Обнаружение аномалий:** Использование VAE для выявления аномалий в наборе данных, например, искаженных или неправильно классифицированных изображений.


Шаг 1: **Установка необходимых библиотек**

Устанавливаем необходимые библиотеки для работы с данными, обучением модели и визуализацией результатов. Эти библиотеки предоставляют инструменты для работы с тензорами, загрузки и предобработки данных, обучения моделей и построения графиков. Используем torch и torchvision для работы с нейронными сетями, matplotlib — для визуализации, а numpy — для работы с массивами данных.

In [ ]:
!pip install torch torchvision matplotlib numpy

Шаг 2: **Импорт библиотек**

Импортируем необходимые модули и классы из установленных библиотек. Эти модули и классы используются для создания и обучения модели, работы с данными и визуализации результатов. Используем стандартные импорты для удобства и совместимости с другими частями кода.

In [ ]:
import torch
from torch import nn
from torch.distributions import Independent, Normal, Bernoulli
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms
import os
from torchvision.datasets import MNIST
import numpy as np

Шаг 3: **Установка фиксированного seed для воспроизводимости**

Устанавливаем фиксированный seed для всех случайных процессов. Это обеспечивает воспроизводимость результатов экспериментов, что важно для сравнения различных моделей и настроек. Используем torch.manual_seed для CPU и torch.cuda.manual_seed_all для GPU, а также настраиваем параметры cudnn для детерминированного режима.

In [ ]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

Шаг 4: **Определение пользовательского класса Dataset**

Создаем пользовательский класс для загрузки данных из сохраненных файлов. Этот класс позволяет работать с данными, сохраненными в формате .pt, и применять к ним трансформации. Наследуемся от torch.utils.data.Dataset для удобства использования с DataLoader.

In [ ]:
class TransposedMNISTDataset(Dataset):
    def __init__(self, file_path: str, transform=None):
        self.data = torch.load(file_path, weights_only=False)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Шаг 5: **Определение трансформаций для данных**

Определяем последовательность трансформаций для предобработки изображений. Трансформации нужны для нормализации изображений и их преобразования в бинарные значения. Используем ToTensor для преобразования изображений в тензоры, view для преобразования в одномерный вектор и Lambda для бинаризации.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1)),
    transforms.Lambda(lambda x: (x > 0.5).float()),
])

Шаг 6: **Проверка наличия файлов и пересохранение данных**

Проверяем наличие файлов с данными и пересохраняем их, если они отсутствуют или некорректны. Это гарантирует, что данные всегда будут загружены и сохранены в нужном формате. Используем os.path.exists для проверки наличия файлов и torch.save для сохранения данных.

In [ ]:
train_file_path = 'transposed_mnist_train.pt'
test_file_path = 'transposed_mnist_test.pt'

if os.path.exists(train_file_path):
    os.remove(train_file_path)
if os.path.exists(test_file_path):
    os.remove(test_file_path)

print("Файлы не найдены или имеют некорректную форму. Пересохраняем данные...")

original_train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
original_test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

train_data = [(img, label) for img, label in original_train_dataset]
test_data = [(img, label) for img, label in original_test_dataset]

torch.save(train_data, 'transposed_mnist_train.pt')
torch.save(test_data, 'transposed_mnist_test.pt')

print("Данные успешно пересохранены.")

Шаг 7: **Проверка наличия файлов и создание экземпляров Dataset**

Проверяем наличие файлов с данными и создаем экземпляры пользовательского класса Dataset. Это позволяет использовать данные для обучения и тестирования модели. Используем os.path.exists для проверки наличия файлов и создаем экземпляры TransposedMNISTDataset для работы с данными.

In [ ]:
if not os.path.exists(train_file_path):
    print(f"Файл {train_file_path} не найден.")
else:
    print(f"Файл {train_file_path} найден.")
    train_dataset = TransposedMNISTDataset(file_path=train_file_path, transform=None)
    print(f"Количество образцов в обучающем наборе: {len(train_dataset)}")

if not os.path.exists(test_file_path):
    print(f"Файл {test_file_path} не найден.")
else:
    print(f"Файл {test_file_path} найден.")
    test_dataset = TransposedMNISTDataset(file_path=test_file_path, transform=None)
    print(f"Количество образцов в тестовом наборе: {len(test_dataset)}")

Шаг 8: **Проверка содержимого файлов**

Проверяем типы и размеры данных в загруженных файлах. Это помогает убедиться, что данные загружены правильно и имеют ожидаемую структуру. Используем torch.load для загрузки данных и выводим информацию о типах и размерах.

In [ ]:
if os.path.exists(train_file_path):
    train_data = torch.load(train_file_path, weights_only=False)
    print(f"Тип данных в train_data: {type(train_data)}")
    print(f"Тип первого образца: {type(train_data[0][0])}")
    print(f"Размер первого образца: {train_data[0][0].size()}")
    print(f"Тип первой метки: {type(train_data[0][1])}")
    print(f"Значение первой метки: {train_data[0][1]}")

if os.path.exists(test_file_path):
    test_data = torch.load(test_file_path, weights_only=False)
    print(f"Тип данных в test_data: {type(test_data)}")
    print(f"Тип первого образца: {type(test_data[0][0])}")
    print(f"Размер первого образца: {test_data[0][0].size()}")
    print(f"Тип первой метки: {type(test_data[0][1])}")
    print(f"Значение первой метки: {test_data[0][1]}")

Шаг 9: **Определение параметров модели**

Определяем параметры модели, такие как размер латентного пространства, скрытого слоя и размер входного изображения. Эти параметры влияют на архитектуру модели и её способность к обучению. Выбираем значения, которые обеспечивают баланс между сложностью модели и вычислительными ресурсами.

In [ ]:
d, nh, D = 32, 200, 28 * 28  # Размер латентного пространства, скрытого слоя, размер входного изображения

Шаг 10: **Определение устройства (GPU или CPU)**

Определяем устройство для выполнения вычислений (GPU или CPU). Это позволяет использовать доступные вычислительные ресурсы для ускорения обучения модели. Используем torch.device для автоматического выбора устройства.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Шаг 11: **Определение энкодера и декодера**

Определяем архитектуру энкодера и декодера. Эти компоненты модели необходимы для кодирования входных данных в латентное пространство и декодирования обратно в исходное пространство. Используем последовательные слои Linear и ReLU для создания нейронных сетей, а Sigmoid для нормализации выхода декодера.

In [ ]:
enc = nn.Sequential(
    nn.Linear(D, nh),
    nn.ReLU(),
    nn.Linear(nh, nh),
    nn.ReLU(),
    nn.Linear(nh, 2 * d)  # 2 * d для mu и log_var
).to(device)

dec = nn.Sequential(
    nn.Linear(d, nh),
    nn.ReLU(),
    nn.Linear(nh, nh),
    nn.ReLU(),
    nn.Linear(nh, D),
    nn.Sigmoid()  # Добавляем Sigmoid для нормализации выхода в диапазон [0, 1]
).to(device)

Шаг 12: **Инициализация весов**

Инициализируем веса нейронных сетей. Корректная инициализация весов помогает ускорить обучение и предотвращает проблемы с затуханием или взрывом градиентов. Используем xavier_uniform_ для инициализации весов и constant_ для инициализации смещений.

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)

enc.apply(init_weights)
dec.apply(init_weights)

Шаг 13: **Определение класса VAE**

Определяем класс VAE, который объединяет энкодер и декодер. Этот класс позволяет легко использовать модель для обучения и инференса. Наследуемся от torch.nn.Module и определяем методы для кодирования, декодирования и прямого прохода.

In [ ]:
class VAE(nn.Module):
    def __init__(self, enc, dec):
        super(VAE, self).__init__()
        self.enc = enc
        self.dec = dec

    def encode(self, x):
        h = self.enc(x)
        mean = h[:, :d]
        logvar = h[:, d:]
        return mean, logvar

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        generator = torch.Generator().manual_seed(seed)
        eps = torch.randn(std.size(), device=mean.device, generator=generator)
        z = mean + eps * std
        return z

    def decode(self, z):
        logits = self.dec(z)
        return logits

    def forward(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        logits = self.decode(z)
        return logits, mean, logvar

Шаг 14: **Функция потерь ELBO с использованием Bernoulli распределения**

Определяем функцию потерь ELBO для обучения VAE. Функция потерь ELBO используется для минимизации различий между реконструкцией и оригинальным изображением, а также для регуляризации латентного пространства. Используем Independent для работы с многомерными распределениями и kl_divergence для вычисления KL-дивергенции.

In [ ]:
def loss_vae(x, vae):
    batch_size = x.size(0)
    recon_probs, mean, logvar = vae(x)

    epsilon = 1e-7
    recon_probs = torch.clamp(recon_probs, min=epsilon, max=1 - epsilon)

    pz = Independent(Normal(loc=torch.zeros(batch_size, d).to(x.device),
                            scale=torch.ones(batch_size, d).to(x.device)),
                     reinterpreted_batch_ndims=1)
    qz_x = Independent(Normal(loc=mean,
                              scale=torch.exp(0.5 * logvar)),
                       reinterpreted_batch_ndims=1)

    px_z = Independent(Bernoulli(probs=recon_probs),
                       reinterpreted_batch_ndims=1)

    log_px_z = px_z.log_prob(x).sum(dim=-1)
    kl_div = torch.distributions.kl_divergence(qz_x, pz).sum(dim=-1)
    elbo = log_px_z - kl_div

    return -elbo.mean(), recon_probs

Шаг 15: **Функция генерации образцов из латентного пространства**

Определяем функцию для генерации новых образцов из латентного пространства. Эта функция позволяет создавать новые изображения цифр, основываясь на случайных точках в латентном пространстве. Используем torch.randn для генерации случайных точек и sigmoid для нормализации выхода.

In [ ]:
def sample_vae(dec, n_samples=50):
    with torch.no_grad():
        samples = torch.sigmoid(dec(torch.randn(n_samples, d).to(device)))
        samples = samples.view(n_samples, 28, 28).cpu().numpy()
    return samples

Шаг 16: **Функция визуализации образцов**

Определяем функцию для визуализации сгенерированных образцов. Эта функция позволяет визуально оценить качество генерируемых изображений. Используем matplotlib для создания сетки изображений.

In [ ]:
def plot_samples(samples, h=5, w=10):
    fig, axes = plt.subplots(nrows=h,
                             ncols=w,
                             figsize=(int(1.4 * w), int(1.4 * h)),
                             subplot_kw={'xticks': [], 'yticks': []})
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(samples[i], cmap='gray')

Шаг 17: **Функция визуализации реконструкций**

Определяем функцию для визуализации реконструкций тестовых образцов. Эта функция позволяет оценить качество восстановления изображений моделью. Используем matplotlib для создания сетки изображений оригиналов и реконструкций.

In [ ]:
def plot_reconstructions(vae, test_loader, n_samples=25):
    with torch.no_grad():
        data, _ = next(iter(test_loader))
        data = data[:n_samples].to(device)
        recon_probs, _, _ = vae(data)
        recon_probs = torch.sigmoid(recon_probs)
        recon_probs = recon_probs.view(n_samples, 28, 28).cpu().numpy()
        data = data.view(n_samples, 28, 28).cpu().numpy()

        fig, axes = plt.subplots(nrows=n_samples // 5, ncols=10, figsize=(14, 7),
                                 subplot_kw={'xticks': [], 'yticks': []})
        for i in range(n_samples):
            if i % 5 == 0:
                axes[i // 5, 2 * (i % 5)].set_title("Orig")
                axes[i // 5, 2 * (i % 5) + 1].set_title("Recon")
            axes[i // 5, 2 * (i % 5)].imshow(data[i], cmap='gray')
            axes[i // 5, 2 * (i % 5) + 1].imshow(recon_probs[i], cmap='gray')
        plt.tight_layout()
        plt.show()

Шаг 18: **Функция обучения модели**

Определяем функцию для обучения модели VAE. Эта функция выполняет обучение модели на обучающем наборе данных и оценивает её качество на тестовом наборе данных. Используем torch.optim.Adam для оптимизации параметров модели и DataLoader для загрузки данных в батчах.

In [ ]:
def train_model(loss_fn, model, train_loader, test_loader, num_epochs, learning_rate=1e-3):
    vae = VAE(model[0], model[1]).to(device)
    optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate, weight_decay=1e-5)

    train_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        vae.train()
        total_train_loss = 0
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            loss, _ = loss_fn(data, vae)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        vae.eval()
        total_test_loss = 0
        with torch.no_grad():
            for batch_idx, (data, _) in enumerate(test_loader):
                data = data.to(device)
                loss, _ = loss_fn(data, vae)
                total_test_loss += loss.item()

        avg_test_loss = total_test_loss / len(test_loader)
        test_losses.append(avg_test_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')

    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.show()

    plot_reconstructions(vae, test_loader, n_samples=25)

    vae.eval()
    with torch.no_grad():
        sample_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)
        for i, (data, labels) in enumerate(sample_loader):
            data = data.to(device)
            mean, logvar = vae.encode(data)
            break

        plt.figure(figsize=(10, 10))
        plt.scatter(mean[:, 0].cpu().numpy(), mean[:, 1].cpu().numpy(), c=labels.cpu().numpy(), cmap='tab10')
        plt.colorbar(label='Digit Label')
        plt.xlabel('Latent Dimension 1')
        plt.ylabel('Latent Dimension 2')
        plt.title('Latent Space Visualization')
        plt.show()

    vae.eval()
    with torch.no_grad():
        z = torch.randn(50, d, device=device)
        generated_images = vae.decode(z)

        samples = generated_images.view(50, 28, 28).cpu().numpy()
        plot_samples(samples, h=5, w=10)

Шаг 19: **Использование DataLoader**

Создаем экземпляры DataLoader для загрузки данных в батчах. Это позволяет эффективно загружать и перемешивать данные для обучения и тестирования. Используем DataLoader с указанием размера батча и режима перемешивания.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Шаг 20: **Создание экземпляра VAE**

Создаем экземпляр модели VAE, объединяя энкодер и декодер. Это позволяет легко передавать модель в функции для обучения и инференса. Создаем список с энкодером и декодером для удобства передачи в функции.

In [ ]:
model = [enc, dec]

Шаг 21: **Обучение модели**

Обучаем модель VAE на заданном количестве эпох. Это позволяет моделью научиться реконструировать изображения и генерировать новые цифры. Используем функцию train_model с заданными параметрами для обучения модели.

In [ ]:
train_model(loss_vae, model=model, train_loader=train_loader, test_loader=test_loader, num_epochs=10)

**Заключение**

В данном проекте мы создали и обучили вариационный автокодировщик для реконструкции цифр из датасета MNIST. Модель показала хорошие результаты в восстановлении изображений и демонстрирует потенциал для дальнейшего исследования латентного пространства и генерации новых изображений.